In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.cache import SQLiteCache
from langchain.chains import LLMChain
from langchain.globals import set_llm_cache

set_llm_cache(SQLiteCache("challenge-12.db"))

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie": "Spider Man",
        "answer": "🕷️🕸️🏙️",
    },
    {
        "movie": "Little Women",
        "answer": "👭📖🎨",
    },
    {
        "movie": "Lalaland",
        "answer": "🎶💃🌟",
    },
    {
        "movie": "Top Gun",
        "answer": "✈️🏍️🔥",
    },
    {
        "movie": "Me Before You",
        "answer": "💑💔🦽",
    },
    {
        "movie": "Harry Porter",
        "answer": "⚡🧙‍♂️🏰",
    }
]

example_prompt = PromptTemplate.from_template("Human: {movie}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: Describe the movie {movie} in three emojis.",
    input_variables=["movie"],
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) |  prompt | llm






In [4]:
def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [5]:
invoke_chain("Thor")

content='AI: ⚡🔨🌩️'


In [7]:
invoke_chain("Truman Show")

content='AI: 🌎🎥👁️'


In [8]:
invoke_chain("What is the movie I asked about first?")

content='AI: 🕷️🕸️🏙️'
